In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)

,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_17_9_8,0.997584,0.993598,0.996861,0.995118,0.000452,0.001497,0.000693,0.001119,"Hidden Size=[45], regularizer=0.01, learning_r..."
1,model_17_9_7,0.997575,0.994248,0.996867,0.995471,0.000454,0.001345,0.000692,0.001038,"Hidden Size=[45], regularizer=0.01, learning_r..."
2,model_17_9_9,0.997572,0.992979,0.996850,0.994778,0.000454,0.001641,0.000696,0.001196,"Hidden Size=[45], regularizer=0.01, learning_r..."
3,model_17_9_10,0.997545,0.992395,0.996834,0.994455,0.000459,0.001778,0.000699,0.001270,"Hidden Size=[45], regularizer=0.01, learning_r..."
4,model_17_9_6,0.997537,0.994924,0.996864,0.995836,0.000461,0.001187,0.000692,0.000954,"Hidden Size=[45], regularizer=0.01, learning_r..."
5,model_17_9_11,0.997506,0.991846,0.996816,0.994151,0.000466,0.001906,0.000703,0.001340,"Hidden Size=[45], regularizer=0.01, learning_r..."
6,model_17_9_12,0.997461,0.991335,0.996797,0.993866,0.000475,0.002026,0.000707,0.001405,"Hidden Size=[45], regularizer=0.01, learning_r..."
7,model_17_9_5,0.997459,0.995619,0.996851,0.996205,0.000475,0.001024,0.000695,0.000869,"Hidden Size=[45], regularizer=0.01, learning_r..."
8,model_17_9_13,0.997411,0.990859,0.996776,0.993599,0.000484,0.002137,0.000712,0.001466,"Hidden Size=[45], regularizer=0.01, learning_r..."
9,model_17_9_14,0.997359,0.990419,0.996755,0.993352,0.000494,0.002240,0.000716,0.001523,"Hidden Size=[45], regularizer=0.01, learning_r..."
